In [6]:
import numpy as np
import pandas as pd
import pickle

In [44]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [45]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 150)

In [46]:
mbd = 'microbusiness_density'

df_train, df_test, df_subm = utils.load_dataset(subm='../submission/submission_2023-02-28_18_21_26.csv')
df_all, df_census = utils.merge_dataset(df_train, df_test, pop=False, unemploy=False, census=False, co_est=False, coord=False, fix_pop=True, outlier=True, outlier_method='v2')
df_all = utils.merge_scale41(df_all, df_subm, df_census)

/tmp/working/code-analysis/../modules/utils.py:298: DtypeWarning: Columns (102) have mixed types.Specify dtype option on import or set low_memory=False.
  df_census = load_census(BASE)


smooth_outlier: max_scale=40
used method: v2
# of fixed cfips: 1210
# of fixed value: 2476


In [47]:
params={
        "act_thre": 140,
        "abs_thre": 0,
        "USE_LAG": 7,
        "USE_TREND": True,
        "blacklist": [],
        "blacklistcfips": [],
        "clip": (-0.0044, 0.0046),
        "model": 'lgb',
        "max_window": 12,
        "light": True,
        "start_max_scale": 41
    }

## バリデーション

### 前の予測結果を用いて予測していく

In [37]:
# accum_pred = models.LgbmBaseline('params_accum',df_subm, df_all, df_census, start_all_dict=35, params=params)
# accum_pred.accum_validation(m_len=2, max_pred_m=1)

## Submissionの作成

### 予測結果を活用して予測

In [48]:
test_subm = models.LgbmBaseline('test_subm_accum', df_subm, df_all, df_census, start_all_dict=40, params=params)
test_subm.create_submission(target_scale=[42, 43, 44, 45], save=False)

add lag features: max_scale=40
created df_all_dict[40]
add lag features: max_scale=41
created df_all_dict[41]
valid_times:  42
pred_m:  1
train_times:  41
use df_all_dict[41]
['scale', 'state_i', 'select_rate1_lag1', 'select_rate1_lag2', 'select_rate1_lag3', 'select_rate1_lag4', 'select_rate1_lag5', 'select_rate1_lag6', 'select_rate1_lag7', 'select_rate1_rsum2', 'select_rate1_rsum4', 'select_rate1_rsum6', 'select_rate1_rsum8', 'select_rate1_rsum10', 'select_rate1_rsum12', 'select_active_lag1_diff1', 'select_active_lag1_diff2', 'select_active_lag1_diff3', 'select_active_lag1_diff4', 'select_active_lag1_diff5', 'select_active_lag1_diff6', 'select_active_lag1_diff7']
use lgb.
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] min_data_in_leaf is set=213, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=213
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=

KeyError: 'select_lastactive41'

In [49]:
df = test_subm.df_all_dict[41]
df

,cfips,county,state,microbusiness_density,active,year,month,scale,state_i,mbd_origin,select_lastactive30,select_lastmbd30,select_lastactive31,select_lastmbd31,select_lastactive32,select_lastmbd32,select_lastactive33,select_lastmbd33,select_lastactive34,select_lastmbd34,select_lastactive35,select_lastmbd35,select_lastactive36,select_lastmbd36,select_lastactive37,select_lastmbd37,select_lastactive38,select_lastmbd38,select_lastactive39,select_lastmbd39,select_lastactive40,select_lastmbd40,select_lastactive41,select_lastmbd41,select_rate1,select_active_lag1,select_mbd_lag1,select_mbd_origin_lag1,select_active_lag2,select_mbd_lag2,select_mbd_origin_lag2,select_active_lag3,select_mbd_lag3,select_mbd_origin_lag3,select_active_lag4,select_mbd_lag4,select_mbd_origin_lag4,select_active_lag5,select_mbd_lag5,select_mbd_origin_lag5,select_active_lag6,select_mbd_lag6,select_mbd_origin_lag6,select_active_lag7,select_mbd_lag7,select_mbd_origin_lag7,select_active_lag1_diff1,select_active_lag1_diff2,select_active_lag1_diff3,select_active_lag1_diff4,select_active_lag1_diff5,select_active_lag1_diff6,select_active_lag1_diff7,select_rate1_lag1,select_rate1_lag2,select_rate1_lag3,select_rate1_lag4,select_rate1_lag5,select_rate1_lag6,select_rate1_lag7,select_rate1_rsum2,select_rate1_rsum4,select_rate1_rsum6,select_rate1_rsum8,select_rate1_rsum10,select_rate1_rsum12
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10001_2019-08-01,10001,0,Delaware,5.184650,6624.0,2019,8,0,0,5.199344,8766.0,6.352220,8895.0,6.445699,8973.0,6.502221,8945.0,6.481931,9126.0,6.613092,9420.0,6.826137,9458.0,6.853673,9486.0,6.873963,9618.0,6.969616,9708.0,7.034833,9784.0,7.089906,9795.0,7.097703,0.000000,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10001_2019-09-01,10001,0,Delaware,5.237020,6672.0,2019,9,1,0,5.237020,8766.0,6.352220,8895.0,6.445699,8973.0,6.502221,8945.0,6.481931,9126.0,6.613092,9420.0,6.826137,9458.0,6.853673,9486.0,6.873963,9618.0,6.969616,9708.0,7.034833,9784.0,7.089906,9795.0,7.097703,0.010101,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10001_2019-10-01,10001,0,Delaware,5.306093,6760.0,2019,10,2,0,5.306093,8766.0,6.352220,8895.0,6.445699,8973.0,6.502221,8945.0,6.481931,9126.0,6.613092,9420.0,6.826137,9458.0,6.853673,9486.0,6.873963,9618.0,6.969616,9708.0,7.034833,9784.0,7.089906,9795.0,7.097703,0.013189,6672.0,5.237020,5.237020,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,48.0,48.0,NaN,NaN,NaN,NaN,NaN,0.010101,0.000000,0.000000,0.0,0.0,0.0,0.0,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101
10001_2019-11-01,10001,0,Delaware,5.333566,6795.0,2019,11,3,0,5.333566,8766.0,6.352220,8895.0,6.445699,8973.0,6.502221,8945.0,6.481931,9126.0,6.613092,9420.0,6.826137,9458.0,6.853673,9486.0,6.873963,9618.0,6.969616,9708.0,7.034833,9784.0,7.089906,9795.0,7.097703,0.005178,6760.0,5.306093,5.306093,6672.0,5.237020,5.237020,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,6624.0,5.184650,5.199344,88.0,136.0,136.0,NaN,NaN,NaN,NaN,0.013189,0.010101,0.000000,0.0,0.0,0.0,0.0,0.023290,0.023290,0.023290,0.023290,0.023290,0.023290
10001_2019-12-01,10001,0,Delaware,5.293535,6744.0,2019,12,4,0,5.293535,8766.0,6.352220,8895.0,6.445699,8973.0,6.502221,8945.0,6.481931,9126.0,6.613092,9420.0,6.826137,9458.0,6.853673,9486.0,6.873963,9618.0,6.969616,9708.0,7.034833,9784.0,7.089906,9795.0,7.097

In [27]:
df_submission = pd.read_csv('../submission/test_subm_accum_2023-02-25_21:02:46.csv', index_col='row_id')
df_merged = utils.compare_submission(df_submission, 'submission_19346')

In [28]:
df_merged['smape'].mean()

0.009844264000484165

In [29]:
# ensemble
0.01261112205424169

# trend_multi有り
0.03200090765098272

0.03200090765098272

In [52]:
target = 'a' if False else 'b'

In [53]:
target

'b'

In [13]:
dic.get('low_trend_params').get(4).get(params)

AttributeError: 'NoneType' object has no attribute 'get'